# FASTTEXT

In [1]:
import pandas as pd
import numpy as np
import json
import spacy
import stopwordsiso as stopwords
from sklearn.manifold import TSNE
import re
import fasttext
import fasttext.util

In [2]:
# !python -m spacy download pl_core_news_sm

In [3]:
nlp = spacy.load("pl_core_news_sm")
polish_stopwords = stopwords.stopwords("pl")

In [4]:
file_path = 'data/fragments_classification.jsonl'  
data = pd.read_json(file_path, lines=True)
data.head()

,text,label
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[[0, 8, odwrócenie]]"
1,@USER No właśnie o tym jest ten tweet 😄,[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[[23, 38, wzmocnienie]]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[]


In [5]:
def adjust_labels_to_words(row):
    text = row['text']
    labels = row['label']
    adjusted_labels = []
    
    for start_idx, end_idx, label in labels:
        while start_idx > 0 and text[start_idx-1].isalnum():
            start_idx -= 1
        
        while end_idx < len(text) and text[end_idx].isalnum():
            end_idx += 1
        
        adjusted_labels.append((start_idx, end_idx, label))
    
    return adjusted_labels



def preprocess_text(text):
    text = re.sub(r'@\w+', '', text)

    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower().strip()

    doc = nlp(text)

    # return [token.lemma_ for token in doc if token.lemma_.lower() not in polish_stopwords]
    return [token.lemma_ for token in doc]



def get_labels(row):
    text = row['text']
    return [(preprocess_text(text[start_idx : end_idx]), label) for start_idx, end_idx, label in row['label']]

In [6]:
data['label'] = data.apply(adjust_labels_to_words, axis = 1)
data['tokens'] = data['text'].apply(preprocess_text)
data['labels2'] = data.apply(get_labels, axis = 1)


data.head()

,text,label,tokens,labels2
0,Nie uzna gola. Robben był kilka metrów w polu ...,"[(0, 8, odwrócenie)]","[nie, uzny, gol, robben, być, kilka, metr, w, ...","[([nie, uzny], odwrócenie)]"
1,@USER No właśnie o tym jest ten tweet 😄,[],"[no, właśnie, o, to, być, ten, tweet]",[]
2,@USER @USER Widać chcą wiecej polskich mord go...,"[(23, 38, wzmocnienie)]","[widać, chcieć, wiecej, polski, mord, gotujacy...","[([wiecej, polski], wzmocnienie)]"
3,"Idę spać bo padam na twarz, w końcu w domuuuu",[],"[iść, spać, bo, padać, na, twarz, w, koniec, w...",[]
4,@USER Tak się poznałam z moim chłopakiem 😂 cza...,[],"[tak, się, poznać być, z, mój, chłopak, , cza...",[]


In [7]:
with open('data/full_text_classification.jsonl', 'r', encoding='utf-8') as file:
    texts = [preprocess_text(json.loads(line)['text']) for line in file]
texts.extend(data['tokens'])

In [8]:
# fasttext.util.download_model('pl', if_exists = 'ignore')

ft = fasttext.load_model('cc.pl.300.bin')

In [12]:
fragment_texts = set()
fragments = []
for labels in data['labels2']:
    for label in labels:
        text = ' '.join(label[0])
        print(text)
        if text in fragment_texts:
            continue
        fragment_texts.add(text)
        try:
            fragments.append((text, ft.get_word_vector(text), label[1]))
        except:
            pass

nie uzny
wiecej polski
nie decydować
nie mieć
nie obudzić
niemy
pierdoli
więcej wiara
nie znać
trudniejszy zagadka
niby być
nie wstydź
nie zwalnia
odrobina myśleć
nie przyzwyczajony
nie być
chyba żart
trochę oksymorona
nie zawsze
nie grać
więcej problem
bardzo prestiżowy
nabazgrać
nie na
nie każdy
ewidentny faul
nie mieć
zajebiść
fatalnie
super film
nie chcieć
w koniec
trochę czas
won
pisowsko mendo
nie na
prawdziwy mistrz
dzieciak
świetny zabawa
nie kompromitować
nie wiadomo
prymitywny dno
cip
chyba
nie konczyć
nie mieć
nie odpisujć
wielki wrażenie
móc pomóc
musieć wkroczyć
nie sprawdzać
bez media
nie mieć
zajebistego
dobry argument
mały pytanie
nigdy
nie interesować
nie pożyć
musieć być
nienawidzić
najlepsi europoseł
albo raczej
niestety
nie mieć
nie być
wawka
każdy
to nie
bardzo
bez roznica
nie mieć
gówniać problem
göwniany opozycja
gówniać domek
domek
gówno bat
doć
chyba jakiś
żałosny człowiek
jedynie
a nie
w koniec
chyba promocja
brawo za
trochę inaczej
weć z
weć
oczywiście
niecał

[('nie uzny',
  array([-2.82148402e-02, -9.88406129e-03,  1.91110652e-03, -1.25954300e-02,
         -5.66795608e-03,  1.13291722e-02, -4.31530550e-03,  2.01552059e-03,
         -1.46032893e-03,  2.32611056e-02, -6.01442717e-03, -4.57145227e-03,
          2.74532381e-03, -1.62095239e-03,  3.83535656e-03,  2.30531059e-02,
          6.12309203e-03,  6.30218443e-03,  3.58805503e-03, -1.01389270e-03,
          2.26131752e-02, -1.05824182e-03, -1.87402274e-02, -5.98744815e-03,
         -8.24333634e-03,  2.47361548e-02, -7.21406844e-03, -7.51133915e-03,
         -2.98140850e-03, -3.15525103e-05,  6.33057905e-03, -1.96133624e-03,
         -1.58748366e-02,  2.53719296e-02, -2.21573422e-03,  7.95146637e-03,
         -3.02457321e-03, -1.28431022e-02, -1.57216005e-02,  5.10234293e-03,
          6.12631766e-03,  5.69525734e-03, -3.74110113e-03, -4.24440913e-02,
          8.98198970e-03, -7.30762025e-03, -4.96583711e-03,  2.88204080e-03,
          1.06784087e-02, -5.12294983e-03,  1.91269517e-02, -4

## Model

In [13]:
df_fragments = pd.DataFrame(fragments, columns=['text', 'vector', 'label'])

In [16]:
import plotly.express as px

tsne = TSNE(n_components=2, random_state=42, perplexity=10)
tsne_results = tsne.fit_transform(np.vstack(df_fragments['vector'].values))

tsne_df = pd.DataFrame(tsne_results, columns=['x', 'y'])
tsne_df['label'] = df_fragments['label']
tsne_df['text'] = df_fragments['text']

fig = px.scatter(tsne_df, x='x', y='y', color='label', 
                 title='Wizualizacja osadzeń Fasttext przy użyciu t-SNE',
                 labels={'label': 'Wydźwięk'},
                 hover_name=tsne_df['text'], 
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers'))

fig.show()